In [1]:
import numpy as np
from functools import reduce


class Node:
    def __init__(self, name):
        self.name = name
        self.cardinality = None
        self.likelihood = None
        self.priors = None
        self.belief = None
        self.parents = []
        self.children = []
        self.m = None

    def add_parent(self, node):
        self.parents.append(node)
        node.children.append(self)

    def __str__(self):
        return self.name

    def message_to_parent(self, parent):
        """
        returns marginalized out parent message:
            - in m: group all entries by receiver parent values (all with 0 together, all with 1 together)
            - use other values in groups to get likelihood and messages from other parents
            - multiply those values in each group element
            - sum each group
        """
        likelihood = self.get_likelihood()
        parents_priors = np.array([p.message_to_child(self)
                                   for p in self.parents
                                   if p != parent])
        parent_i = self.parents.index(parent)

        stack = np.vstack([np.dot(self.m.take(r, axis=parent_i).transpose(),
                                  parents_priors.prod(axis=0))
                           for r in range(parent.cardinality)])

        return np.dot(stack, likelihood)

    def message_to_child(self, child):
        children_messages = np.array([c.message_to_parent(self)
                                      for c in self.children
                                      if c != child])
        if len(children_messages) > 0:
            unnormalized = (children_messages * self.get_priors()).prod(axis=0)
            message = unnormalized/unnormalized.sum()
            return message
        return self.get_priors()

    def get_likelihood(self):
        if self.likelihood is not None:
            return self.likelihood

        incoming_children_messages = np.array([c.message_to_parent(self)
                                               for c in self.children])
        return incoming_children_messages.prod(axis=0)

    def get_priors(self):
        # Formula 2
        # Union sur la partition des intersections des parents
        if self.priors is not None:
            return self.priors

        parents_messages = [p.message_to_child(self)
                            for p in self.parents]
        priors = reduce(np.dot, [self.m.transpose()]+parents_messages)
        return priors

    def get_belief(self):
        if self.belief is not None:
            return self.belief

        unnormalized = self.get_likelihood() * self.get_priors()
        return unnormalized/unnormalized.sum()

def dot_T(x, y):
    return np.dot(x.transpose(), y)

In [11]:
rain = Node("rain")
rain.cardinality = 2
rain.priors = np.array([1, 0]) #  no=0 yes=1

sprinkler = Node("sprinkler")
sprinkler.cardinality = 2
sprinkler.priors = np.array([0.9, 0.1]) #  no=0 yes=1

m = np.zeros((2, 2, 2)) #  rain, sprinkler, holmes' grass
m[1, 1, 1] = 1
m[0, 1, 1] = 0.9
m[0, 1, 0] = 0.1
m[1, 0, 1] = 1
m[0, 0, 0] = 1
holmes = Node("holmes")
holmes.cardinality = 2
holmes.m = m

m = np.zeros((2, 2)) # rain, watson's grass
m[1, 1] = 1
m[0, 1] = 0.2
m[0, 0] = 0.8
watson = Node("watson")
watson.cardinality = 2
watson.m = m


holmes.add_parent(rain)
holmes.add_parent(sprinkler)
watson.add_parent(rain)

Holmes' grass is wet, don't know about Watson

In [12]:
holmes.likelihood = np.array([0, 1])
watson.likelihood = np.array([1, 1])
print("Belief about Watson's:", watson.get_belief())

Belief about Watson's: [0.8 0.2]


In [19]:
sprinkler.get_belief()

array([0., 1.])

In [26]:
holmes.get_priors()
# watson.get_priors()

# # Holmes grass is wet

# holmes.message_to_parent(rain)
# holmes.message_to_parent(sprinkler)


# # Watson's grass is also wet

# watson.likelihood = np.array([0, 1])

# sprinkler.get_belief()

array([0.728, 0.272])

In [27]:
holmes.get_belief()

array([0., 1.])